In [5]:
# Spleen Proximity project
import SimpleITK as sitk
import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob
import numpy as np
import monai
import random

from monai.utils import first, set_determinism
from monai.transforms import (
    Activations,
    CenterSpatialCrop,
    EnsureChannelFirst,
    RandRotate90,
    Resize,
    ResizeWithPadOrCrop,
    AsDiscrete,
    Compose,
    LoadImage,
    RandFlip,
    RandRotate,
    RandZoom,
    RandAdjustContrast,
    RandShiftIntensity,
    RandScaleIntensity,
    RandHistogramShift,
    ScaleIntensity,
    ScaleIntensityRange,
    AsDiscrete,
    Activationsd,
    AsDiscreted,
    EnsureChannelFirstd,
    Compose,
    CropForegroundd,
    Crop,
    LoadImaged,
    Orientationd,
    RandCropByPosNegLabeld,
    SaveImaged,
    ScaleIntensity,
    ScaleIntensityRanged,
    ScaleIntensityRangePercentiles,
    SpatialCrop,
    ThresholdIntensity,
    Spacingd,
    Invertd,
    Spacing
)
from monai.handlers.utils import from_engine
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.metrics import DiceMetric
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset, DataLoader, Dataset, decollate_batch, ImageDataset, DistributedWeightedRandomSampler
from monai.config import print_config
from monai.apps import download_and_extract
from sklearn.metrics import balanced_accuracy_score

pin_memory = torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [7]:
# path to dataset folder
data_dir = '/home/mohammad/projects/ErrolProjects/spleen/dataNifti/abnormal/cropped_segmentor/'
class_names = sorted(x for x in os.listdir(data_dir)
                     if os.path.isdir(os.path.join(data_dir, x)))
class_names = ['AAST13_old', 'AAST45_old', 'negative_old']
num_class = len(class_names)
image_files = [
    [
        os.path.join(data_dir, class_names[i], x)
        for x in random.Random(0).sample(os.listdir(os.path.join(data_dir, class_names[i])), min(608, len(os.listdir(os.path.join(data_dir, class_names[i])))))
    ]
    for i in range(num_class)
]

num_each = [len(image_files[i]) for i in range(num_class)]
image_files_list = []
image_class = []
for i in range(num_class):
    image_files_list.extend(image_files[i])
    image_class.extend([i] * num_each[i])
num_total = len(image_class)


print(f"Total image count: {num_total}")
print(f"Label names: {class_names}")
print(f"Label counts: {num_each}")

Total image count: 1216
Label names: ['AAST13_old', 'AAST45_old', 'negative_old']
Label counts: [455, 153, 608]
